*Tester usado: DT380B*

<img src="https://github.com/brunoricciotti/ME1/blob/main/LABORATORIO/LABORATORIO%201/img/error_DT380B.png?raw=true:, width=100" alt="My Image" width=500>

https://www.viaindustrial.com/manuales_pdf/multimetro-digital-economico-basico-96830-dt830-b-ebchq-manual-espanol.pdf

**Patron HP 3468b **

https://xdevs.com/doc/HP_Agilent_Keysight/HP%203468A,%20B%20Operating.pdf

<img src="https://github.com/brunoricciotti/ME1/blob/main/LABORATORIO/LABORATORIO%201/img/error_HP3468B.png?raw=true:, width=100" alt="My Image" width=500>


In [5]:
import statistics as st
import numpy as np
import math as mt

mA = 0.001
uA = 0.000001

def incertidumbre_A(X) -> float:
  return st.stdev(X)/mt.sqrt(len(X))

#Esta funcion es para error en cuentas. Si nos dan el error
#fijo, usamos la formula, adaptando la manera en la que le enviamos los argumentos
def inecrtidumbre_B(er_p, nd, Ci, Xm) -> float:
  return Xm*((er_p/100)+(nd/Ci))/mt.sqrt(3)

def incertidumbre_total_A_B(uA, uB) -> float:
  return mt.sqrt( uA**2 + uB**2 )

def truncate(number, decimals=0):
    """
    Returns a value truncated to a specific number of decimal places.
    """
    if not isinstance(decimals, int):
        raise TypeError("decimal places must be an integer.")
    elif decimals < 0:
        raise ValueError("decimal places has to be 0 or more.")
    elif decimals == 0:
        return mt.trunc(number)

    factor = 10.0 ** decimals
    return mt.trunc(number * factor) / factor

#**Escala: 2mA**
#**Fuente: 100uA**

In [15]:
V_ver = [0.00010,0.00010,0.00010,0.00009,0.00009]
V_med = [114,115,115,114,115]#uA
V_ver_m = st.mean(V_ver)
V_med_m = st.mean(V_med)
ecal_m = V_med_m - V_ver_m
print(f'V_ver_m: {V_ver_m}\nV_med_m: {V_med_m}\necal_m: {ecal_m}')

V_ver_m: 9.6e-05
V_med_m: 114.6
ecal_m: 114.599904


##Incertidumbres de tipo A

In [16]:
V_med = list(map(lambda x: x*uA, V_med))#paso de uA a Ampere
uA_V_ver = incertidumbre_A(V_ver)
uA_V_med = incertidumbre_A(V_med)
print(f'Incertidumbre tipo A\nuA_V_ver: {uA_V_ver}\nuA_V_med: {uA_V_med}')

Incertidumbre tipo A
uA_V_ver: 2.4494897427831775e-06
uA_V_med: 2.449489742783171e-07


##Incertidumbres de tipo B


Al calcular las cuentas ci para ambas mediciones, vemos que EN ESTE CASO PARTICULAR todas las medidiones que nos dan son de 3 digitos. Como vemos que en ambos casos el valor mas probable dio con un solo digito no decimal, multiplicamos por 100 ambos valores medios, teniendo 3 digitos no decimales, y al hacer int() los numeros decimamles se truncan.

In [23]:
#valores sacados del datasheet del tester:
nd_ver = 6
er_p_ver = 0.17
Ci_V_ver = int(V_ver_m*100000) #son 6 digitos, y V_ver_m esta con 5 decimales

nd_med = 2
er_p_med = 1
Ci_V_med = int(V_med_m*1) #son 3 digitos, y V_med_m esta en uA sin decimales

uB_V_ver = inecrtidumbre_B(er_p_ver, nd_ver, Ci_V_ver, V_ver_m)
uB_V_med = inecrtidumbre_B(er_p_med, nd_med, Ci_V_med, V_med_m*uA)
print(f'Incertidumbre tipo B\nuB_V_ver: {uB_V_ver}\nuB_V_med: {uB_V_med}')

Incertidumbre tipo B
uB_V_ver: 3.70446407920678e-05
uB_V_med: 1.8224213181251904e-06


##Incertidumbre total de cada magnitud


In [24]:
uc_V_ver = uB_V_ver
uc_V_med = incertidumbre_total_A_B(uA_V_med, uB_V_med)
print(f'Incertidumbre total\nuc_V_ver: {uc_V_ver}\nuc_V_med: {uc_V_med}')

Incertidumbre total
uc_V_ver: 3.70446407920678e-05
uc_V_med: 1.8388092507808297e-06


##Incertidumbre combinada - funcion error de calculo

In [25]:
uc_ecal = mt.sqrt(uc_V_ver**2+uc_V_med**2)
print(f'uc_ecal: {uc_ecal}')

uc_ecal: 3.7090249808731276e-05


##Factor de expansion

Podemos ver que las incertidumbres de tipo A son mucho mayores, ambos casos uA >> uB, por lo menos por 10 veces. Por lo tanto usamos la tabla para k B dominante

In [26]:
relacion_ui = (uA_V_ver + uA_V_med) / (uB_V_ver + uB_V_med)

print(f'relacion_ui: {relacion_ui}')

relacion_ui: 0.06932447606722689


In [27]:
k95 = 1.9 #relacion es aprox 0,70

##Incertidumbre expandida

In [28]:
Uc_ecal = uc_ecal*k95
print(f'Uc_ecal: {Uc_ecal}')

Uc_ecal: 7.047147463658943e-05


## verificacion calibracion

Para que la calibracion sea aceptabe, $|e_{cal_m}|+Uc_{ecal} <= \Delta V$

In [30]:
delta_V = V_med_m*uA*((er_p_med/100)+(nd_med/Ci_V_med))
print(f'delta_V: {delta_V}')

suma = abs(ecal_m*uA)+ Uc_ecal
print(f'|ecal_m|+Uc_ecal = {suma}')

if suma <= delta_V: print('Verifica calibracion')
else: print('NO verifica calibracion')

delta_V: 3.1565263157894737e-06
|ecal_m|+Uc_ecal = 0.0001850713786365894
NO verifica calibracion
